# LLM-Based D&D Simulation Tutorial

This notebook demonstrates how to use the LLM-based D&D simulation system to create AI-driven gameplay sessions based on real human campaign data.

## Overview

The simulation system includes:
- **Campaign Parameter Extraction**: Analyze human campaigns to extract initialization parameters
- **Character Creation**: Generate D&D characters with personalities and classes
- **Turn Sampling**: Create realistic turn sequences using various sampling methods
- **Character Agents**: LLM-powered characters with memory and decision-making
- **Game Session Management**: Orchestrate complete D&D scenarios

## Prerequisites

1. Install required packages: `pip install anthropic`
2. Set your Anthropic API key: `export ANTHROPIC_API_KEY=your_key_here`
3. Ensure you have access to the human campaign data files

## Setup and Imports

In [1]:
import os
import sys
import json
import random
from pathlib import Path

# Add the llm_scaffolding directory to the Python path
sys.path.append(str(Path.cwd().parent / "llm_scaffolding"))

# Import our simulation system
from dnd_simulation import (
    extract_campaign_parameters,
    create_characters,
    sample_turn_sequence,
    CharacterAgent,
    GameSession
)

# Set random seed for reproducible results
random.seed(42)

print("✅ Setup complete!")

✅ Setup complete!


## API Key Setup (Secure Method)

We'll load the API key from a secure file that's excluded from version control.

**First-time setup:**
1. Open the `api_key.txt` file in the project root directory
2. Replace the placeholder text with your actual Anthropic API key
3. Save the file (it's already in .gitignore so it won't be committed)

**Alternative methods:**
- Environment variable: `export ANTHROPIC_API_KEY=your_key_here`
- Direct assignment (not recommended): `api_key = 'your_key_here'`

In [2]:
def load_api_key():
    """Load API key from secure file or environment variable."""

    # Method 1: Try to load from secure file
    api_key_file = Path.cwd().parent / "api_key.txt"

    if api_key_file.exists():
        try:
            with open(api_key_file, 'r') as f:
                content = f.read().strip()

                # Skip comment lines and empty lines
                lines = [
                    line.strip() for line in content.split('\n')
                    if line.strip() and not line.strip().startswith('#')
                ]

                if lines and lines[0] != 'PASTE_YOUR_ANTHROPIC_API_KEY_HERE':
                    api_key = lines[0]
                    print("✅ API key loaded from api_key.txt")
                    print(f"Key prefix: {api_key[:10]}...")
                    return api_key
                else:
                    print(
                        "⚠️  api_key.txt found but contains placeholder text")
                    print(
                        "Please edit api_key.txt and paste your actual API key"
                    )
        except Exception as e:
            print(f"❌ Error reading api_key.txt: {e}")

    # Method 2: Try environment variable
    api_key = os.getenv('ANTHROPIC_API_KEY')
    if api_key:
        print("✅ API key loaded from environment variable")
        print(f"Key prefix: {api_key[:10]}...")
        return api_key

    # Method 3: No key found
    print("❌ No API key found!")
    print("\n🔧 Setup options:")
    print(
        "1. RECOMMENDED: Edit api_key.txt in the project root and paste your key"
    )
    print(
        "2. Set environment variable: export ANTHROPIC_API_KEY=your_key_here")
    print("3. Get your API key from: https://console.anthropic.com/")

    return None

# Load the API key
api_key = load_api_key()
os.environ['ANTHROPIC_API_KEY'] = api_key 

if not api_key:
    print("\n⚠️  Cannot proceed without API key. Please set up your key and rerun this cell.")
else:
    print("\n🚀 Ready to proceed with simulation!")

✅ API key loaded from api_key.txt
Key prefix: sk-ant-api...

🚀 Ready to proceed with simulation!


## Step 1: Extract Campaign Parameters

Let's load a real human campaign and extract parameters that we can use to initialize our simulation.

In [ ]:
# Choose a campaign file to analyze
campaign_file = "../data/raw-human-games/individual_campaigns/10391-guardians-of-gridori.json"

# Extract parameters from the human campaign
print("📊 Extracting campaign parameters...")
campaign_params = extract_campaign_parameters(campaign_file)

print(f"\n🎯 Campaign Parameters:")
print(f"  Campaign Name: {campaign_params['campaign_name']}")
print(f"  Total Messages: {campaign_params['total_messages']:,}")
print(f"  Number of Players: {campaign_params['num_players']}")
print(f"  Character Names: {campaign_params['character_names']}")

# We'll use this player count for our simulation
num_players = campaign_params['num_players']
print(f"\n🎮 Using {num_players} players for simulation")

📊 Extracting campaign parameters...

🎯 Campaign Parameters:
  Campaign Name: 10391-guardians-of-gridori
  Total Messages: 216
  Number of Players: 5
  Character Names: ['Dungeon Master' 'Jinx' 'Faen' 'Dmitrei' 'Thokk']
  Character Personalities: {'Dungeon Master': 'The Dungeon Master exhibits a patient and descriptive style, providing rich details about the environment and events unfolding. They maintain a fair and engaging approach, responding thoughtfully to player actions and driving the narrative forward.', 'Jinx': "Jinx's player seems confident and assertive, taking charge in dangerous situations while also showing concern for the group's well-being. They exhibit a practical mindset, prioritizing the mission's objectives and encouraging the party to stay focused.", 'Faen': "Faen's player has a more reserved and introspective demeanor, often reflecting on their character's thoughts and emotions. They display a curious nature, asking insightful questions and engaging with the storyt

In [ ]:
for i in range(campaign_params['num_players']):
    print(f"  Character Personalities: {campaign_params['character_personalities']['Dungeon Master']}")

  Character Personalities: The Dungeon Master exhibits a patient and descriptive style, providing rich details about the environment and events unfolding. They maintain a fair and engaging approach, responding thoughtfully to player actions and driving the narrative forward.
  Character Personalities: The Dungeon Master exhibits a patient and descriptive style, providing rich details about the environment and events unfolding. They maintain a fair and engaging approach, responding thoughtfully to player actions and driving the narrative forward.
  Character Personalities: The Dungeon Master exhibits a patient and descriptive style, providing rich details about the environment and events unfolding. They maintain a fair and engaging approach, responding thoughtfully to player actions and driving the narrative forward.
  Character Personalities: The Dungeon Master exhibits a patient and descriptive style, providing rich details about the environment and events unfolding. They maintain a f

## Step 2: Create Characters

Now let's create our AI characters using the player count from the human campaign.

In [5]:
# Create characters based on the campaign parameters
print("🧙 Creating characters...")

# For now, we'll use default characters (future versions can extract from campaign)
characters = create_characters(num_players=num_players)

print(f"\n✨ Created {len(characters)} characters:")
for char in characters:
    print(f"  🗡️  {char.name} ({char.character_class}): {char.personality}")

# Get character names for turn sampling
character_names = [char.name for char in characters]
print(f"\n👥 Character roster: {character_names}")

🧙 Creating characters...

✨ Created 5 characters:
  🗡️  Aria (Fighter): Brave and decisive warrior who leads by example and protects her allies.
  🗡️  Thorin (Rogue): Cunning and stealthy scout who prefers shadows and clever solutions.
  🗡️  Gandalf (Wizard): Wise and scholarly spellcaster who seeks knowledge and magical solutions.
  🗡️  Lyra (Cleric): Compassionate healer who provides spiritual guidance and divine magic.
  🗡️  Zara (Ranger): Nature-loving archer who tracks enemies and protects the wilderness.

👥 Character roster: ['Aria', 'Thorin', 'Gandalf', 'Lyra', 'Zara']


## Step 3: Generate Turn Sequence

Create a realistic turn sequence using uniform sampling across all characters.

In [10]:
# Generate a turn sequence for our mini-session
total_turns = 12  # Adjust for desired session length

print(f"🎲 Generating turn sequence ({total_turns} turns)...")
turn_sequence = sample_turn_sequence(
    character_names=character_names,
    total_turns=total_turns,
    method='uniform'
)

print(f"\n📋 Turn sequence: {turn_sequence}")

# Show turn distribution
from collections import Counter
turn_counts = Counter(turn_sequence)
print(f"\n📊 Turn distribution:")
for char_name, count in turn_counts.items():
    print(f"  {char_name}: {count} turns ({count/total_turns*100:.1f}%)")

🎲 Generating turn sequence (12 turns)...

📋 Turn sequence: ['Aria', 'Aria', 'Gandalf', 'Gandalf', 'Aria', 'Gandalf', 'Lyra', 'Aria', 'Lyra', 'Gandalf', 'Thorin', 'Aria']

📊 Turn distribution:
  Aria: 5 turns (41.7%)
  Gandalf: 4 turns (33.3%)
  Lyra: 2 turns (16.7%)
  Thorin: 1 turns (8.3%)


## Step 4: Initialize Game Session

Create the game session manager that will orchestrate our D&D simulation.

In [11]:
# Create game session
print("🎭 Initializing game session...")
game_session = GameSession(characters=characters, api_key=api_key)

print(f"✅ Game session created with {len(characters)} characters")
print(f"📝 Game log initialized (currently {len(game_session.game_log)} events)")

🎭 Initializing game session...
✅ Game session created with 4 characters
📝 Game log initialized (currently 0 events)


## Step 5: Define the Scenario

Set up an engaging D&D scenario for our AI characters to navigate.

In [12]:
# Define the scenario
scenario = """You're adventurers who just entered a mysterious tavern called 'The Whispering Griffin' in the border town of Millhaven. The tavern is dimly lit by flickering candles, and the air is thick with the scent of ale and wood smoke. 

The bartender, a nervous-looking halfling named Pip, keeps glancing toward the stairs leading to the upper rooms. In the far corner, a cloaked figure sits alone, whispering urgently to someone unseen in the shadows. 

You notice that despite the late hour, the tavern is unusually quiet - only a few patrons sit scattered at tables, and they all seem to be avoiding eye contact. Something feels off about this place.

What do you do?"""

print("🏰 Scenario set:")
print(f"\n{scenario}")
print("\n" + "="*60)

🏰 Scenario set:

You're adventurers who just entered a mysterious tavern called 'The Whispering Griffin' in the border town of Millhaven. The tavern is dimly lit by flickering candles, and the air is thick with the scent of ale and wood smoke. 

The bartender, a nervous-looking halfling named Pip, keeps glancing toward the stairs leading to the upper rooms. In the far corner, a cloaked figure sits alone, whispering urgently to someone unseen in the shadows. 

You notice that despite the late hour, the tavern is unusually quiet - only a few patrons sit scattered at tables, and they all seem to be avoiding eye contact. Something feels off about this place.

What do you do?



## Step 6: Run the Simulation

Now let's run the complete D&D simulation and watch our AI characters interact!

In [13]:
# Run the simulation
print("🚀 Starting D&D simulation...")
print("\n" + "="*60)

# Execute the scenario
game_session.run_scenario(initial_scenario=scenario, turn_sequence=turn_sequence)

🚀 Starting D&D simulation...

=== D&D SIMULATION STARTING ===
Scenario: You're adventurers who just entered a mysterious tavern called 'The Whispering Griffin' in the border town of Millhaven. The tavern is dimly lit by flickering candles, and the air is thick with the scent of ale and wood smoke. 

The bartender, a nervous-looking halfling named Pip, keeps glancing toward the stairs leading to the upper rooms. In the far corner, a cloaked figure sits alone, whispering urgently to someone unseen in the shadows. 

You notice that despite the late hour, the tavern is unusually quiet - only a few patrons sit scattered at tables, and they all seem to be avoiding eye contact. Something feels off about this place.

What do you do?
Characters: ['Aria', 'Thorin', 'Gandalf', 'Lyra']
Total turns: 12

Aria: *I grip the hilt of my sword and survey the tavern warily, taking in every detail. Lowering my voice, I address my companions:* 

"Keep your wits about you, friends. There's a strange tension 

## Step 7: Analyze the Results

Let's examine the game log and see what our AI characters accomplished.

In [14]:
# Display the complete game log
print("📖 Complete Game Log:")
print("=" * 60)

for i, event in enumerate(game_session.game_log, 1):
    print(f"Turn {event['turn_number']:2d} | {event['character']:8s} | {event['action']}")
    if i % 5 == 0:  # Add spacing every 5 turns
        print()

print("\n" + "=" * 60)

📖 Complete Game Log:
Turn  1 | Aria     | *I grip the hilt of my sword and survey the tavern warily, taking in every detail. Lowering my voice, I address my companions:* 

"Keep your wits about you, friends. There's a strange tension in this place that sets me ill at ease. Pip seems anxious, and who knows what that cloaked figure in the corner is up to."

*I glance meaningfully at the lone patron before continuing in a firm tone:* "For now, let's take a table near the exit, just to be safe. If any trouble arises, I'll be ready." 

*Nodding resolutely, I lead the way to an empty table with a clear view of the door, my hand never leaving my weapon's grip. Once seated, I keep a vigilant watch over the tavern's occupants, ready to leap into action should any threat reveal itself.*
Turn  2 | Aria     | *I stride into the tavern, hand resting on the hilt of my sword, scanning the room with a watchful gaze. Turning to my companions, I speak in a low but resolute tone.*

"Keep your wits about 

## Step 8: Character Memory Analysis

Let's see how each character's memory evolved during the session.

In [15]:
# Display character memories
print("🧠 Character Memory States:")
print("=" * 60)

for character in characters:
    print(f"\n{character.name} ({character.character_class}):")
    print(f"Memory: {character.memory_summary}")
    print("-" * 40)

🧠 Character Memory States:

Aria (Fighter):
Memory: Here is my updated memory summary as Aria the Fighter after the recent events:

I remained vigilant, weapon at the ready, as Gandalf and Lyra sought to cajole the innkeeper Pip into revealing any nefarious forces at play within the ominous tavern. Though they took a measured diplomatic approach initially, the air was tense with the implicit threat of our combined might should Pip prove deceptive or complicit with darker powers. 

Lyra placed a calming hand on my arm, tempering my eagerness to meet any hostilities with swift force. She implored me to embody wisdom through empathy while still issuing a firm warning to Pip - speak truthfully to receive our aid in vanquishing evil, or face the full wrath of Lyra's radiant glory without mercy. Her holy symbol flared brilliantly, the divine light both reassuring and foreboding.

Gandalf too demonstrated his formidable wizardly powers, raising his staff to create an aura of preternatural pro

## Step 9: Session Statistics

Generate some basic statistics about the simulation session.

In [16]:
# Generate session statistics
from collections import Counter
from datetime import datetime

print("📊 Session Statistics:")
print("=" * 40)

# Turn statistics
character_turns = Counter(event['character'] for event in game_session.game_log)
print(f"Total turns: {len(game_session.game_log)}")
print(f"Total characters: {len(characters)}")
print(f"Average turns per character: {len(game_session.game_log) / len(characters):.1f}")

print("\n👥 Character participation:")
for char_name, turn_count in character_turns.most_common():
    percentage = (turn_count / len(game_session.game_log)) * 100
    print(f"  {char_name}: {turn_count} turns ({percentage:.1f}%)")

# Action length statistics
action_lengths = [len(event['action']) for event in game_session.game_log]
print(f"\n📝 Response statistics:")
print(f"  Average response length: {sum(action_lengths) / len(action_lengths):.1f} characters")
print(f"  Shortest response: {min(action_lengths)} characters")
print(f"  Longest response: {max(action_lengths)} characters")

print("\n✨ Session complete!")

📊 Session Statistics:
Total turns: 12
Total characters: 4
Average turns per character: 3.0

👥 Character participation:
  Aria: 5 turns (41.7%)
  Gandalf: 4 turns (33.3%)
  Lyra: 2 turns (16.7%)
  Thorin: 1 turns (8.3%)

📝 Response statistics:
  Average response length: 960.8 characters
  Shortest response: 725 characters
  Longest response: 1224 characters

✨ Session complete!


## Step 10: Export Results (Optional)

Save the simulation results for further analysis.

In [17]:
# Export simulation results
simulation_results = {
    'campaign_params': campaign_params,
    'characters': [
        {
            'name': char.name,
            'character_class': char.character_class,
            'personality': char.personality,
            'final_memory': char.memory_summary
        } for char in characters
    ],
    'scenario': scenario,
    'turn_sequence': turn_sequence,
    'game_log': game_session.game_log,
    'session_stats': {
        'total_turns': len(game_session.game_log),
        'character_turns': dict(character_turns),
        'avg_response_length': sum(action_lengths) / len(action_lengths)
    }
}

# Save to file
output_file = f"llm_simulation_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_file, 'w') as f:
    json.dump(simulation_results, f, indent=2)

print(f"💾 Results saved to: {output_file}")
print(f"📁 File size: {os.path.getsize(output_file)} bytes")

💾 Results saved to: llm_simulation_results_20250717_170632.json
📁 File size: 21509 bytes


## Conclusion

🎉 **Congratulations!** You've successfully run a complete LLM-based D&D simulation!

### What we accomplished:

1. **✅ Parameter Extraction**: Analyzed a real human campaign to extract player count and structure
2. **✅ Character Creation**: Generated AI characters with distinct personalities and classes
3. **✅ Turn Sampling**: Created a realistic turn sequence using uniform sampling
4. **✅ Memory Management**: Each character maintained context and memory across turns
5. **✅ Interactive Gameplay**: Characters responded to situations and each other naturally
6. **✅ Session Management**: Complete game log with timestamps and turn tracking

### Key Features Demonstrated:

- **Multi-agent LLM system**: Each character has their own Claude client
- **Memory persistence**: Characters remember past events and update their knowledge
- **Turn-based interaction**: Realistic D&D-style turn sequence
- **Extensible architecture**: Easy to add new sampling methods, character extraction, etc.
- **Real campaign integration**: Parameters derived from actual human gameplay

### Next Steps:

- **🔮 Extract character data**: Pull actual character names/classes from human campaigns
- **📈 Advanced sampling**: Implement activity-based or weighted turn sampling
- **🧠 Enhanced memory**: Add verbatim recent events + summarized older events
- **🎪 Richer scenarios**: Create more complex, multi-scene adventures
- **📊 Analysis tools**: Compare AI gameplay patterns to human campaigns

The simulation framework is now ready for more sophisticated experiments with emergent AI behavior in D&D settings!